# Demonstrating the dataset generation for an ampl model

This notebook shows the generation of the dataset that are subsequently used to train a neural network for optimal control.

In [1]:
# !pip install numpy
# !pip install matplotlib
# !pip install joblib
# !pip install progressbar 
# !pip install amplpy
# !pip install tqdm
# !pip install ampl
### %pylab notebook
from pyquad.ekin import*
from tqdm import tqdm
import numpy as np

In [3]:
%pip install -q amplpy
from amplpy import AMPL, ampl_notebook
ampl = ampl_notebook(
    modules=["coin", "snopt", "highs", "cbc", "gurobi", "cplex"], # pick from over 20 modules including most commercial and open-source solvers
    license_uuid="d6d79098-3902-4752-9af1-f51b6b24800b") # your license UUID (e.g., free ampl.com/ce or ampl.com/courses licenses)

Note: you may need to restart the kernel to use updated packages.
Licensed to AMPL Academic Community Edition License for <a.elarrassi@tudelft.nl>.


In [4]:
from amplpy import AMPL

ampl = AMPL()

# Set a solver — choose one that's installed
ampl.setOption('solver', 'ipopt')  # or 'ipopt', 'gurobi', etc.

# Simple model
ampl.eval("""
param a;
var x;
maximize obj: a * x;
s.t. constraint: x <= 10;
""")

# Set parameter value
ampl.getParameter('a').set(2)

# Solve
solved = ampl.solve()

# Output
print(solved)
print("Solved:", ampl.getValue("solve_result"))
print("x =", ampl.getValue("x"))
print("Objective:", ampl.getObjective("obj").value())


Ipopt 3.12.13: 

******************************************************************************
This program contains Ipopt, a library for large-scale nonlinear optimization.
 Ipopt is released as open source code under the Eclipse Public License (EPL).
         For more information visit http://projects.coin-or.org/Ipopt
******************************************************************************

This is Ipopt version 3.12.13, running with linear solver mumps.
NOTE: Other linear solvers might be more efficient (see Ipopt documentation).

Number of nonzeros in equality constraint Jacobian...:        0
Number of nonzeros in inequality constraint Jacobian.:        0
Number of nonzeros in Lagrangian Hessian.............:        0

Total number of variables............................:        1
                     variables with only lower bounds:        0
                variables with lower and upper bounds:        0
                     variables with only upper bounds:        1
Tot

In [5]:
from amplpy import AMPL, Environment
import matplotlib.pyplot as plt
import numpy as np

# Load AMPL model
ampl = AMPL()

# Optionally specify the AMPL environment if needed
# ampl = AMPL(Environment('/path/to/ampl'))

# Load your model file
ampl.read(ampl_mod_path)

# Set key parameters
ampl.param['n'] = 99
ampl.param['epsilon'] = 0.01
ampl.param['tn'] = 2.0

# Optional: Set initial and final conditions (example)
ampl.param['dx0'] = 0.0
ampl.param['dy0'] = 0.0
ampl.param['dz0'] = 0.0
ampl.param['vxn'] = 1.0  # desired x velocity at final time

# Solve
ampl.solve()

# Fetch results
timegrid = ampl.get_variable('timegrid').to_dict()
dx = ampl.get_variable('dx').to_dict()
dy = ampl.get_variable('dy').to_dict()
dz = ampl.get_variable('dz').to_dict()

# Convert to arrays for plotting
t = np.array([timegrid[i] for i in sorted(timegrid)])
x = np.array([dx[i] for i in sorted(dx)])
y = np.array([dy[i] for i in sorted(dy)])
z = np.array([dz[i] for i in sorted(dz)])

# Plot trajectory
fig = plt.figure()
ax = fig.add_subplot(111, projection='3d')
ax.plot(x, y, z, label='Quadrotor Trajectory')
ax.set_xlabel('X [m]')
ax.set_ylabel('Y [m]')
ax.set_zlabel('Z [m]')
ax.set_title('3D Trajectory of Quadrotor')
ax.legend()
plt.show()


NameError: name 'ampl_mod_path' is not defined

In [12]:
ampl_mod_path = "ampl/bebop_model_new.mod"

num = 1000

# INITIAL CONDITIONS IN WORLD COORDINATES
x0 = np.random.uniform(-5.0, 5.0, num)
y0 = np.random.uniform(-5.0, 5.0, num)
z0 = np.random.uniform(-1.0, 1.0, num)

vx0 = np.random.uniform(-0.5,0.5, num)
vy0 = np.random.uniform(-0.5,0.5, num)
vz0 = np.random.uniform(-0.5,0.5, num)

phi0   = np.random.uniform(-40, 40, num)*np.pi/180
theta0 = np.random.uniform(-40, 40, num)*np.pi/180
psi0   = np.random.uniform(-180, 180, num)*np.pi/180

p0     = np.random.uniform(-1., 1., num)
q0     = np.random.uniform(-1., 1., num)
r0     = np.random.uniform(-1., 1., num)

utau0  = np.random.uniform(0, 1, [num, 4])

Mx_ext = 0*np.random.uniform(-.04, .04, num)
My_ext = 0*np.random.uniform(-.04, .04, num)
Mz_ext = 0*np.random.uniform(-.01, .01, num)

def transform(x,y,z,vx,vy,vz, phi,theta,psi):
    Rx = np.array([[1, 0, 0], [0, np.cos(phi), -np.sin(phi)], [0, np.sin(phi), np.cos(phi)]])
    Ry = np.array([[np.cos(theta), 0, np.sin(theta)], [0, 1, 0], [-np.sin(theta), 0, np.cos(theta)]])
    Rz = np.array([[np.cos(psi), -np.sin(psi), 0], [np.sin(psi), np.cos(psi), 0], [0, 0, 1]])
    R = Rz@Ry@Rx
    dx, dy, dz = -R.T@[x, y, z]
    vx, vy, vz =  R.T@[vx,vy,vz]
    return dx, dy, dz, vx, vy, vz

# IN BODY COORDINATES
dx0, dy0, dz0, vx0, vy0, vz0 = np.vectorize(transform)(x0, y0, z0, vx0, vy0, vz0, phi0, theta0, psi0)

parameters = {
        "dx0"    : dx0,
        "dy0"    : dy0, 
        "dz0"    : dz0,
        "vx0"    : vx0,
        "vy0"    : vy0,
        "vz0"    : vz0,
        "phi0"   : phi0,
        "theta0" : theta0,
        "psi0"   : psi0,
        "p0"     : p0,
        "q0"     : q0,
        "r0"     : r0,
        "utau0"  : utau0,
        "Mx_ext": Mx_ext,
        "My_ext": My_ext,
        "Mz_ext": Mz_ext
}


# FINAL CONDITIONS
dxn = 0.
dyn = 0.
dzn = 0.

vxn = 0.
vyn = 0
vzn = 0.

phin = 0.
thetan = 0.
psin = 0.

pn = 0.
qn = 0.
rn = 0.

fixed_parameters = {
    "dxn"         : dxn,
    "dyn"         : dyn, 
    "dzn"         : dzn,
    "vxn"         : vxn,
    "vyn"         : vyn,
    "vzn"         : vzn,
    "phin"        : phin,
    "thetan"      : thetan,
    "psin"        : psin,
    "pn"          : pn,
    "qn"          : qn,
    "rn"          : rn,
    "epsilon"     : 1.,
    "n"           : 100,
    "omega_max"   : 9500,
    "omega_min"   : 5500
}

In [13]:
def run_ampl_model(param_dict, file_path):
    print("here")
    model = AMPLModel(ampl_mod_path)
    # model = AMPL()
    # model.read(ampl_mod_path)
    all_params = {**param_dict, **fixed_parameters}
    model.setParameterValues(all_params)
    # model.setParameterValues(fixed_parameters)
    model.ampl.setOption('solver', 'ipopt')
    # model.ampl.setOption('solver_options', 'max_iter=500 tol=1e-6')

    solved = model.solve()
    print(solved)
    print("Solve Result:", model.ampl.getValue("solve_result"))
    print("Solve Message:", model.ampl.getValue("solve_message"))
    # print("Solver status:", model.ampl.getValue("solve_result_code"))
    print(model.solve())
    print("Solved:", model.ampl.getValue("solve_result"))
    # solved = model.checkSolved()
    print(f"{file_path} solved: {solved}")
    print(solved)  # Any solver messages/errors
    print("AMPL Output:")
    # print(model.ampl.getOutput())

    print("Solve Message:", model.ampl.getValue('solve_message'))
    soln_vals = model.getSolutionValues()
    obj_vals = model.getObjectiveValues()
    np.savez(file_path, **{
        "Success": solved,
        "t": soln_vals['timegrid'],
        **soln_vals,
        **obj_vals,
        **param_dict,
        **fixed_parameters
    })

from amplpy import AMPL
import numpy as np

def run_ampl_model1(param_dict, fixed_parameters, ampl_mod_path, file_path):
    print("Starting AMPL model run")

    ampl = AMPL()
    ampl.read(ampl_mod_path)

    # Set parameters in AMPL model
    for param_name, value in {**param_dict, **fixed_parameters}.items():
        ampl.getParameter(param_name).set(value)

    ampl.setOption('solver', 'ipopt')
    # ampl.setOption('solver_options', 'max_iter=500 tol=1e-6')

    solved = ampl.solve()

    print("Solve Result:", ampl.getValue("solve_result"))
    print("Solve Message:", ampl.getValue("solve_message"))
    print("Solved:", solved)
    print(f"{file_path} solved: {solved}")

    # Extract solution values - assuming your model defines these sets/variables
    # This depends on your model; here are some generic examples:

    soln_vals = {}
    obj_vals = {}

    # For example, if your model has variables 'timegrid' and objectives
    if 'timegrid' in ampl.getVariableNames():
        soln_vals['timegrid'] = ampl.getVariable('timegrid').getValues().toList()

    # Get objective values
    for obj_name in ampl.getObjectiveNames():
        obj_vals[obj_name] = ampl.getObjective(obj_name).value()

    np.savez(file_path, **{
        "Success": solved,
        **soln_vals,
        **obj_vals,
        **param_dict,
        **fixed_parameters
    })

    print(f"Results saved to {file_path}")


run_ampl_model({key: val[0] for key, val in parameters.items()}, 'test_out')


here
True
Solve Result: solved
Solve Message:  
Ipopt 3.12.13: Optimal Solution Found
True
Solved: solved
test_out solved: True
True
AMPL Output:
Solve Message:  
Ipopt 3.12.13: Optimal Solution Found


In [14]:
def try_different_params(max_attempts=20):
    for attempt in range(max_attempts):
        print(f"Attempt {attempt+1}/{max_attempts}")
        # ... generate parameters as before ...

        # Transform to body coordinates
        dx0, dy0, dz0, vx0_b, vy0_b, vz0_b = np.vectorize(transform)(x0, y0, z0, vx0, vy0, vz0, phi0, theta0, psi0)

        parameters = {
            "dx0": dx0,
            "dy0": dy0,
            "dz0": dz0,
            "vx0": vx0_b,
            "vy0": vy0_b,
            "vz0": vz0_b,
            "phi0": phi0,
            "theta0": theta0,
            "psi0": psi0,
            "p0": p0,
            "q0": q0,
            "r0": r0,
            "utau0": utau0,
            "Mx_ext": Mx_ext,
            "My_ext": My_ext,
            "Mz_ext": Mz_ext
        }
        # print(parameters)

        # Convert single-element arrays to scalars for AMPL
        parameters_fixed = {
            key: val.item() if isinstance(val, np.ndarray) and val.size == 1 else val
            for key, val in parameters.items()
        }
        # Choose index 0 (or random index) for this attempt:
        i = 0
        parameters_fixed = {
            "dx0": dx0[i].item(),
            "dy0": dy0[i].item(),
            "dz0": dz0[i].item(),
            "vx0": vx0_b[i].item(),
            "vy0": vy0_b[i].item(),
            "vz0": vz0_b[i].item(),
            "phi0": phi0[i].item(),
            "theta0": theta0[i].item(),
            "psi0": psi0[i].item(),
            "p0": p0[i].item(),
            "q0": q0[i].item(),
            "r0": r0[i].item(),
            "utau0": utau0[i],  # Assuming utau0 is shape (1000,4), pass the 4-length array for this i
            "Mx_ext": Mx_ext[i].item(),
            "My_ext": My_ext[i].item(),
            "Mz_ext": Mz_ext[i].item()
        }


        try:
            print(parameters_fixed)
            run_ampl_model({key: val[999] for key, val in parameters.items()}, f'test_out_attempt_{attempt}')
            print("Model solved successfully!")
            return  # Exit on first success
        except Exception as e:
            print("Failed to solve model:", e)
            print("Trying next parameters...\n")

    print("All attempts failed. Try adjusting parameter ranges or model constraints.")

# Run the parameter trying function
try_different_params(20)


Attempt 1/20
{'dx0': -0.0021796507595872976, 'dy0': 1.4270149752867922, 'dz0': -0.2671937129041948, 'vx0': 0.22248595411498212, 'vy0': -0.4418718091276435, 'vz0': -0.16007083990629223, 'phi0': 0.2212216383827999, 'theta0': 0.5781525205759003, 'psi0': -0.9740202575745861, 'p0': -0.04923848260304875, 'q0': 0.23590269943762543, 'r0': -0.8464340653421034, 'utau0': array([0.08872825, 0.39488137, 0.48009658, 0.14664805]), 'Mx_ext': 0.0, 'My_ext': -0.0, 'Mz_ext': 0.0}
here
True
Solve Result: solved
Solve Message:  
Ipopt 3.12.13: Optimal Solution Found
True
Solved: solved
test_out_attempt_0 solved: True
True
AMPL Output:
Solve Message:  
Ipopt 3.12.13: Optimal Solution Found
Model solved successfully!


In [15]:
name = 'HOVER_TO_HOVER_NOMINAL'
datafolder = 'datasets/' + name
datafile = 'datasets/' + name + '.npz'

# make folder if it doesn't exist
if not os.path.exists(datafolder):
    os.makedirs(datafolder)

In [ ]:
# Determine which trajectories to solve
index_set = set(range(num)) - {
    int(file.replace('.npz', ''))
    for file in os.listdir(datafolder)
    if file.endswith('.npz') and file.replace('.npz', '').isdigit()
}
# index_set = range(0,999)
print(f"To solve: {len(index_set)} / {num}")

# Solve trajectories in a regular loop
sol_lst = []
for idx in tqdm(sorted(index_set)):
    param_values = {key: val[idx] for key, val in parameters.items()}
    save_path = os.path.join(datafolder, str(idx))
    sol = run_ampl_model(param_values, save_path)
    sol_lst.append(sol)

print('Generated', len(sol_lst), 'trajectories in', datafolder)


In [80]:
def find_failed(datafolder):
    failed = set()
    for file in os.listdir(datafolder):
        if file.endswith('.npz') and file.replace('.npz', '').isdigit():
            idx = int(file.replace('.npz', ''))
            data = np.load(os.path.join(datafolder, file))
            if not data.get('Success', True):  # default True if missing
                failed.add(idx)
                # print(idx)
    return failed

index_set = find_failed(datafolder)
print(index_set)

{0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99, 100, 101, 102, 103, 104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116, 117, 118, 119, 120, 121, 122, 123, 124, 125, 126, 127, 128, 129, 130, 131, 132, 133, 134, 135, 136, 137, 138, 139, 140, 141, 142, 143, 144, 145, 146, 147, 148, 149, 150, 151, 152, 153, 154, 155, 156, 157, 158, 159, 160, 161, 162, 163, 164, 165, 166, 167, 168, 169, 170, 171, 172, 173, 174, 175, 176, 177, 178, 179, 180, 181, 182, 183, 184, 185, 186, 187, 188, 189, 190, 191, 192, 193, 194, 195, 196, 197, 198, 199, 200, 201, 202, 203, 204, 205, 206, 207, 208, 209, 210, 211, 212, 213, 214, 215, 216, 217, 218, 219, 220, 221,

In [81]:
def load_file(file, datafolder):
    try:
        return dict(np.load(datafolder + '/' + file))
    except:
        return None

njobs = 100
trajectory_list = joblib.Parallel(njobs)(
    joblib.delayed(load_file)(file, datafolder) for file in tqdm(os.listdir(datafolder))
)
trajectory_list = [traj for traj in trajectory_list if traj != None]

100%|███████████████████████████████████████████████████████████████████████████████| 1000/1000 [00:03<00:00, 257.41it/s]


In [82]:
keys = trajectory_list[0].keys()
trajectory_list = [traj for traj in trajectory_list if traj.keys()==keys]
keys = keys - {key+'m' for key in keys if key+'m' in keys}

dataset = {key: np.stack([traj[key] for traj in trajectory_list]) for key in tqdm(keys)}

# default parameters
params = AMPLModel(ampl_mod_path).getParameterValues()
dataset = {**params, **dataset}
print(datafile)
# save dataset
np.savez_compressed(datafile, **dataset)

100%|███████████████████████████████████████████████████████████████████████████████████| 73/73 [00:00<00:00, 532.36it/s]


datasets/HOVER_TO_HOVER_NOMINAL.npz


**Filter failed trajectories**

In [83]:
def filter_unsuccesful(dataset):
    return set(np.where(dataset['Success'] == False)[0])

# def filter_unphysical(dataset):
#     unphysical_x = set(np.where(abs(dataset['dx'][:, 1:]-dataset['dx'][:, :-1]) > 0.2)[0])
#     unphysical_y = set(np.where(abs(dataset['dy'][:, 1:]-dataset['dy'][:, :-1]) > 0.2)[0])
#     unphysical_z = set(np.where(abs(dataset['dz'][:, 1:]-dataset['dz'][:, :-1]) > 0.2)[0])
#     return unphysical_x | unphysical_y | unphysical_z

a=np.load(datafile)
num = a['dx'].shape[0]
discard = filter_unsuccesful(a) #| filter_unphysical(a)
keep = set(range(num)) - discard

print(len(discard))
print(num)

print("Unsuccessful trajectories:", len(discard))
print("Total trajectories:", num)
print("Kept trajectories:", len(keep))


if discard:
    b = dict()
    for key in tqdm(a.keys()):
        if len(a[key].shape) > 0:
            if a[key].shape[0] == num:
                b[key] = a[key][list(keep)]
        else:
            b[key] = a[key]
    np.savez_compressed(datafile, **b)

1000
1000
Unsuccessful trajectories: 1000
Total trajectories: 1000
Kept trajectories: 0


100%|█████████████████████████████████████████████████████████████████████████████████| 100/100 [00:00<00:00, 257.12it/s]


**View trajectories**

In [74]:
from amplpy import AMPL

ampl = AMPL()
ampl.eval('param a; let a := 2; display a;')
model = AMPLModel(ampl_mod_path)
model.setParameterValues({key: val[0] for key, val in parameters.items()})

try:
    model.solve()
    print("Solved:", model.checkSolved())
    print("Objective value:", model.getObjectiveValues())
except Exception as e:
    print("AMPL solve failed:", e)


a = 2

AMPL solve failed: 'AMPLModel' object has no attribute 'getLog'


In [8]:
def transform_back(dx,dy,dz,vx,vy,vz,phi,theta,psi):
    Rx = np.array([
        [1, 0, 0],
        [0, np.cos(phi), -np.sin(phi)],
        [0, np.sin(phi), np.cos(phi)]
    ])
    Ry = np.array([
        [np.cos(theta), 0, np.sin(theta)],
        [0, 1, 0],
        [-np.sin(theta), 0, np.cos(theta)]
    ])
    Rz = np.array([
        [np.cos(psi), -np.sin(psi), 0],
        [np.sin(psi), np.cos(psi), 0],
        [0, 0, 1]
    ])
    R = Rz@Ry@Rx
    x_new, y_new, z_new = -R@[dx, dy, dz]
    vx_new, vy_new, vz_new = R@[vx, vy, vz]
    return x_new, y_new, z_new, vx_new, vy_new, vz_new, phi, theta, psi

In [9]:
%matplotlib notebook
a = np.load(datafile)
num = a['dx'].shape[0]
print(num)

t,dx,dy,dz,vx,vy,vz,phi,theta,psi,u = (a[key][0:1000] for key in 't dx dy dz vx vy vz phi theta psi u'.split(' '))

0


In [10]:
import numpy as np
from quadcopter_animation import animation
import importlib
importlib.reload(animation)

x,y,z,vx,vy,vz,phi,theta,psi = np.vectorize(transform_back)(dx,dy,dz,vx,vy,vz,phi,theta,psi)
# t,x,y,z,vx,vy,vz,phi,theta,psi,u = (a[key] for key in 't x y z vx vy vz phi theta psi u'.split(' '))

animation.animate(t, x, y, z, phi, theta, psi, u, multiple_trajectories=True, simultaneous=False, step=1, waypoints=[np.array([0,0,0])], colors=[(255,0,0)]*100)

ValueError: cannot call `vectorize` on size 0 inputs unless `otypes` is set